In [9]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import pandas as pd
from numpy import genfromtxt
import sklearn
import scipy.io

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import calinski_harabasz_score as qmetric
from sklearn.cluster import KMeans

import subprocess
import csv
import sys

In [2]:
class handles_vars:
    def __init__ (self):
        self.Dir = str()
        self.directorio = str()
        self.algoritmo = int()
        self.topic = int()
        self.nametopic = str()
        self.tipo_distancia = str()
        self.clustering = str()
        self.init_order = int()
        self.Rlocation = str()
        self.pca = int()
        self.aleatorio = int()
        self.maximoImagenVista = int()
        self.owa = str()
        self.weightsIt2 = list()
        self.weigthsIt3 = list()
        self.weightsCat2 = list()
        self.weightsCat3 = list()
        self.relevance0 = int()
        self.sinqueries = int()
        self.st = np.ndarray
        self.variances_pca = np.ndarray
        self.ajuste = str()
        self.datosImagenes = np.ndarray
        self.datos = np.ndarray
        self.datosclusters = np.ndarray
        self.topicneg = int()
        self.datosclustervisual = np.ndarray
        self.KmeansCentros = np.ndarray
        self.KmeansDists = np.ndarray
        self.KmeansSumas = np.ndarray
        self.positivas = np.ndarray
        self.queries = np.ndarray
        self.imaRel = np.ndarray
        self.imaRelHistorico = np.ndarray
        self.imaProb = np.ndarray
        self.imaProb1 = np.ndarray
        self.imaProb2 = np.ndarray
        self.imaProbHistoria = np.ndarray
        self.datosyneg = np.ndarray
        self.negativas = np.ndarray
        self.datosyqueries = np.ndarray
        self.datosorden = np.ndarray
        self.datosyqueriesorden = np.ndarray
        self.datosyqueriesynegativasorden = np.ndarray
        self.Ranking = np.ndarray

In [23]:
class InfoCluster:
    def __init__ (self):
        self.id = int()
        self.imagenes = np.ndarray
        self.proba = np.ndarray
        self.numimag = int()
        self.probmax = float()
        self.imagenmax = int()
        self.idxquien = int()
        

In [24]:
def abrir_descriptores_mat(name_fmat):
    descr_mat = scipy.io.loadmat(name_fmat)
    #Convertimos el array n-dimensional en las diferentes listas
    #Names
    names = descr_mat["names"]
    #DatosImagenes
    datosImagenes = descr_mat["datosImagenes"]
    #st
    st = descr_mat["st"]
    #Variances PCA
    variances_pca = descr_mat["variances_pca"]
    return names,datosImagenes,st,variances_pca

In [25]:
def seleccionoDescriptoresNeg(handles,maxnum,tipo):
    # maxnum: número máximo de imágenes negativas a seleccionar
    # tipo: algoritmo de selección de negativas
    # 1: Selecciono las últimas ordenadas por su st
    # 2: Selecciono del listado de imágenes pero de otro topic
    # 3: Selecciono del listado de imágenes
    if tipo == 1:
        fil,col = handles.datos.shape
        inici = fil - maxnum
        descNR = handles.datos[inici:fil]
        
    elif tipo == 2:
        if handles.topic<=70:
            desp = 1
            modulo = 70
        else:
            desp = 71
            modulo = 65
        numero = (handles.topic+1)%modulo + desp #De esta manera, obtengo siempre un topic diferente al nuestro del mismo set 
        handles.topicneg = numero
        
        #Cargamos el nuevo topic
        name_fmat = os.path.join(handles.Dir,"descriptores","topic")
        name_fmat = name_fmat + str(handles.topicneg) + ".mat"
        names,datosImagenes,st,variances_pca = abrir_descriptores_mat(name_fmat)
        
        #ids_images = datosImagenes[:,0]
        descNR = []
        for i in range(maxnum):
            descNR.append(datosImagenes[i,:])
        
        #Los tipos 3 y 4 se necesitan de un listado de imagenes
        return handles,np.array(descNR)

In [26]:
def optimal_k_clusters(descr_image):
    Nclusters_max = 15
    Nrepetitions = 10

    qualities = []
    inertias = []
    models = []
    for k in range(1,Nclusters_max+1):
        kmeans = KMeans(n_clusters=k,
                        init='k-means++', n_init=Nrepetitions,
                        max_iter=500, random_state=2)    
        kmeans.fit(descr_image)
        models.append(kmeans)
        inertias.append(kmeans.inertia_)
        if k >1:
            qualities.append(qmetric(descr_image, kmeans.labels_))
        else:
            qualities.append(0)
    
    best = pd.Series(qualities).idxmax() # get index for the best model
    kmeans = models[best]
    n_clusters = kmeans.get_params()['n_clusters']
    print("\nEl numero optimo de clusters es de: ",n_clusters)
    return kmeans

In [27]:
def GenerarClusteringVisual(handles,descr_imag,metodo,kclusters):
    if metodo == "kmeans18":
        dimension = 50
        descr_redu = descr_imag
        kmeans=KMeans(n_clusters=kclusters,random_state=0).fit(descr_redu)
        #kmeans=KMeans(n_clusters=18,random_state=0).fit(handles.datos[:,2:])
        
        #Calculo la matriz que asigna a cada cluster las imagenes que contiene
        clusterings = kmeans.labels_ 
        
        #Calculo el centro de cada cluster para cada variable
        centroids = kmeans.cluster_centers_ 
        
        #Calculo la distancia de cada imagen al centro de cada cluster (euclidea)
        dists = kmeans.transform(handles.datos[:,2:]) 
        
        #Calculo la suma de las distancias de todos los puntos de un cluster a su centro para cada cluster
        sumas=[]
        for clus in np.unique(clusterings): #Del 0 al 17
                images = np.argwhere(clusterings==clus) #Vector con las imagenes pertenecientes al cluster "clus"
                sumas.append(dists[images,clus].sum()) #Sumamos las distancias de las imagenes pertenecientes al cluster clus hasta el centro de clus
        sumas = np.array(sumas)
        
    elif metodo == "kmeans_optimal":
        descr_redu = descr_imag
        kmeans = optimal_k_clusters(descr_redu)
        
        #Calculo la matriz que asigna a cada cluster las imagenes que contiene
        clusterings = kmeans.labels_ 
        
        #Calculo el centro de cada cluster para cada variable
        centroids = kmeans.cluster_centers_ 
        
        #Calculo la distancia de cada imagen al centro de cada cluster (euclidea)
        dists = kmeans.transform(handles.datos[:,2:]) 
        
        #Calculo la suma de las distancias de todos los puntos de un cluster a su centro para cada cluster
        sumas=[]
        for clus in np.unique(clusterings): #Del 0 al 17
                images = np.argwhere(clusterings==clus) #Vector con las imagenes pertenecientes al cluster "clus"
                sumas.append(dists[images,clus].sum()) #Sumamos las distancias de las imagenes pertenecientes al cluster clus hasta el centro de clus
        sumas = np.array(sumas)
        
    else:
        print("Metodo no valido. Métodos disponibles: 'kmeans18','kmeans_optimal'")
    return clusterings,centroids,dists,sumas

In [28]:
def seleccionImagenesPositivas(handles,tipoCluster,listado_neg):
    # Esta función selecciona 5 imágenes positivas teniendo en cuenta su
    # ranking textual st, i después teniendo en cuenta que no pertenecen al
    # mismo cluster visual para tener imágenes relevantes pero que sean distintas
    
    #Numero de imagenes positivas que selecciono
    num_positivas = 5
    #tipoCluster
    # 1: usamos cluster visual para seleccionar las imagenes positivas
    # 2: usamos cluster textual para seleccionar las imagenes positivas
    # 3: usamos cluster textual + visual para seleccionar las imagenes positivas
    
    if tipoCluster == 1:
        #Tenemos que generar los clusters visuales
        descr_image = handles.datos[:,2:] #Evitamos las dos primeras columnas que son name y name.jpg
        
        numclusters = 18
        
        handles.datosclustervisual,centroids,dists,sumas = GenerarClusteringVisual(handles,descr_image,handles.clustering,kclusters=numclusters)
        identificadores = handles.datos[:,0]
        #En este momento tenemos en datosclustervisual un vector de 300 filas donde a cada imagen le asignamos un cluster.
        
        #Ahora asociamos en una matriz de 300x2 el nombre de la imagen con su cluster
        handles.datosclustervisual = np.c_[identificadores,handles.datosclustervisual]
        handles.KmeansCentros = centroids
        handles.KmeansDists = dists
        handles.KmeansSumas = sumas
        
        imcluster = np.zeros(25) #Trabajamos con 25 clusters textuales, que luego mezclaremos con los 18 visuales
        datoscluster = handles.datosclustervisual
        
    elif tipoCluster == 2:
        datoscluster = handles.datosclusters
        maximo = handles.datosclusters[:,1].max() #Selecciono el valor máximo que representa el número de clusters textuales
        imcluster = np.zeros((1,maximo))
    
    elif tipoCluster == 3:
        num_positivas = num_positivas*2
        
    #Si tenemos listado de imágenes negativas lo usamos para no seleccionar
    # esa imagen como positiva (Existe= 1; No existe=0);
    
    if listado_neg == 0:
        scoresTextual = handles.st[np.argsort(handles.st[:,1])]
        ids = handles.datos[:,0]
        
        datosQueries = []
        positivas = []
        npositivas=0
        i=0
        
        while (npositivas < num_positivas) and i+1 < scoresTextual.shape[0]:
            im1 = int(scoresTextual[i,0]) #Cogemos la primera  (i-esima) imagen del ranking del scoreTextual
            posi = np.argwhere(np.array(datoscluster)==im1)[0][0] #Calculamos su posicion en la matriz que la asocia a su cluster
            idi = int(datoscluster[posi,0]) #Comprobamos que en esa posicion de la matriz tenemos a la imagen
            cluster = int(datoscluster[posi,1]) #Extraemos el cluster al que pertenece
            st = float(scoresTextual[i,2]) #Extraemos la probabilidad de la imagen del cluster en el ranking textual
            pos = int(scoresTextual[i,1]) #Extraemos la posicion en el ranking de pertenecer al cluster
            
            if imcluster[cluster]==0:
                c = 1
                encontrado = 0
                while (c==1 and (not encontrado)):
                    total = np.argwhere(datoscluster[:,1]==cluster)
                    
                    if len(total) >=5:
                        imcluster[cluster] = 1 
                        positivas.append([idi, cluster, pos, st])
                        #Busco los datos de la imagen Query en el listado de imágenes
                        Impos = int(np.argwhere(ids==idi))
                        datosQueries.append(handles.datos[Impos,:])
                        npositivas+=1
                        encontrado=1
                    else:
                        imcluster[cluster] = 1
                    c+=1
                    
                if encontrado:
                    imcluster[cluster]=1
                i+=1
            else:
                i+=1
    else:
        print("No hay listado")
    handles.queries = np.array(datosQueries)
    handles.positivas = np.array(positivas)
    return handles

In [29]:
def CalculoPesosModelo(variances,v_comp):
    variances_norm = variances/variances.sum() #Suma 1
    tam = len(v_comp)
    pesos = np.zeros((tam,1))
    inicio = 0
    for i in range(tam):
        fin = v_comp[i]+inicio
        pesos[i] = variances_norm[inicio:fin].sum()
        inicio = fin +1
    
    pesos = pesos/pesos.sum()
    return pesos

In [12]:
def AjusteMultiplesModelos(v_datos,lista_imag_rel,Rlocation,ajuste):
    if ajuste=="locfit":
        dimension = (lista_imag_rel==1).sum() #Numero de 1 en el vector
        fil_datos,col_datos = v_datos.shape
        inicio = 1
        fin = dimension
        vect_prob = []
        v_componentesModelo = []

        num = -1

        while col_datos > inicio + fin:
            DatosRegresion = np.hstack((v_datos[:,inicio-1:inicio+fin],lista_imag_rel))
            #Guardamos los datos de regresion en un fichero
            np.savetxt("fentrada.txt",DatosRegresion)
            #Ejecutamos el script de R que crea dos ficheros, relevancia.txt y componentes.txt
            #relevancia.txt contiene una matriz con los datos de la regresion 
            #y con los -1 de las neutras convertidos en probabilidades de ser relevantes.
            subprocess.call([Rlocation, '--vanilla', os.getcwd()+r"/generoRelevancia.R"]);

            if 'relevancia.txt' in os.listdir():
                proba_rel = np.array(pd.read_csv(os.getcwd()+r"/relevancia.txt",header=None))
                n_comp = int(np.array(pd.read_csv(os.getcwd()+r"/componentes.txt",header=None)))

                if len(proba_rel) == fil_datos:
                    vect_prob.append(np.array(pd.read_csv(os.getcwd()+r"/relevancia.txt",header=None)))
                    os.remove("relevancia.txt")
                    os.remove("componentes.txt")
                    v_componentesModelo.append(n_comp)
            else:
                cadena =f'No ha sido posible ajustar modelo multiple: {inicio}-{inicio+fin}'
                print(cadena);
                n_comp= fin

            inicio = inicio + n_comp
        
        vect_prob = np.transpose(np.squeeze(np.array(vect_prob)))
        v_componentesModelo = np.transpose(np.squeeze(np.array(v_componentesModelo)))
        
    elif ajuste=="logistic":
        print(f"v_datos: {v_datos.shape}\n\n")
        print(f"lista_imag_rel: {lista_imag_rel.shape}\n\n")
        y_pos = lista_imag_rel[lista_imag_rel==1]
        X_pos = v_datos[np.where(lista_imag_rel==1)[0]]
        y_neg = lista_imag_rel[lista_imag_rel==0]
        X_neg = v_datos[np.where(lista_imag_rel==0)[0]]
        y_test = lista_imag_rel[lista_imag_rel==-1]
        X_test = v_datos[np.where(lista_imag_rel==-1)[0]]
        print(f"X_pos: {X_pos.shape}\n")
        print(f"X_neg: {X_neg.shape}\n")
        print(f"y_pos: {y_pos.shape}\n")
        print(f"y_neg: {y_neg.shape}\n")
        
        X_train = np.vstack((X_pos,X_neg))
        y_train = np.concatenate((y_pos,y_neg))
        print(f"X_train: {X_train.shape}\n")
        print(f"y_train: {y_train.shape}\n")
        model = LogisticRegression(penalty="l2")
        model.fit(X_train,y_train)
        pred = model.predict(X_test)[:,1]
        vect_prob = model.predict_proba(X_test)
        print(f"Probabilidades test: {vect_prob.shape}\n")
        print(vect_prob)
        
        
    return vect_prob,v_componentesModelo

In [31]:
def ClustersConImagenes(VectorClusters):
    CantImagClust = [info.numimag for info in VectorClusters]
    ClusConImg = np.array(CantImagClust) > 0
    numclusters = ClusConImg.sum()
    return numclusters

In [32]:
def ActualizarProbabilidadesClusters(VectorClusters):
    numclusters = len(VectorClusters)
    
    for i in range(numclusters):
        probmax = VectorClusters[i].proba.max()
        idx = VectorClusters[i].proba.argmax()
        VectorClusters[i].probmax = probmax
        imagenmax = VectorClusters[i].imagenes[idx]
        VectorClusters[i].imagenmax = imagenmax
        VectorClusters[i].idxquien = idx
    return VectorClusters

In [33]:
def metodoOrdenarPorClusters(imag_prob,losclusters, umbral):
    VectorClusters = []
    valoresclusters = np.unique(losclusters[:,1])

    for cluster in valoresclusters:
        info_clus = InfoCluster()
        info_clus.id = cluster #Empiezo por el cluster 1
        indices = np.argwhere(losclusters[:,1]==cluster) #Busco los indices que pertecen al cluster
        info_clus.imagenes = losclusters[indices,0] #Selecciono las imagenes que pertenecen al cluster mediante su indice
        
        #Para cada imagen del cluster, busco la probabilidad en el otro vector
        info_clus.proba = np.zeros((len(info_clus.imagenes),1))
        info_clus.numimag = len(info_clus.imagenes)
        
        for j,imagen in enumerate(info_clus.imagenes):
            ids = imagen
            inde_img = np.argwhere(imag_prob[:,0]==ids)
            info_clus.proba[j] = imag_prob[inde_img,1]
        
        probmax = info_clus.proba.max()
        idx = info_clus.proba.argmax()
        info_clus.probmax = probmax
        imagenmax = info_clus.imagenes[idx]
        info_clus.imagenmax = imagenmax
        info_clus.idxquien = idx
        VectorClusters.append(info_clus)
    
    
    #VectorClusters es una lista que contiene objetos de la clase "info_clus", que contienen información acerca de un cluster
    #como las imágenes que pertenecen a ese cluster y sus probabilidades (con eso se puede sacar el maximo y demás cosas)
    
    
    
    Listaordenada = np.array(VectorClusters)
    #print("Tenemos en una lista la información relativa a las imágenes respecto cada cluster")
    #print("POR EJEMPLO:")
    #print(f"Cluster {VectorClusters[0].id}\nTiene {VectorClusters[0].numimag} imágenes que son: {VectorClusters[0].imagenes}")
    #print(f"Las probabilidades de cada imagen de ser relevantes en la búsqueda del Albuquerque Globos son: {VectorClusters[0].proba} ")
    #print(f"Nos guardamos entonces la más probable:\n")
    #print(f"La imagen numero {VectorClusters[0].idxquien} llamada {VectorClusters[0].imagenmax} con una probabilidad de {VectorClusters[0].probmax}")
    
    #Recorro una vez las clusters y asigno una primera imagen de cada cluster
    numclusters = len(Listaordenada)  
    Ranking = []
    continuar = True
    vueltas = 1
    while continuar:
        #Asigno clusters
        #Miro cuantos clusters hay actualmente con imagenes
        numclusters = ClustersConImagenes(Listaordenada)
        i = 1
        seguir = 1
        while seguir:
            #Tomo 1 por cluster
            #Me quedo con un vector con las probabilidades maximas de cada cluster de ser relevantes a la busqueda
            proba_max_clus = np.array([lista.probmax for lista in Listaordenada])
            
            #Me guardo el valor de la probabilidad máxima de entre todos los clusters, su posicion (cluster) y el nombre de la imagen
            maximo = proba_max_clus.max()
            idx = proba_max_clus.argmax()
            idimagen = int(Listaordenada[idx].imagenmax)
            
            #guardo identificador de imagen, probabilidad y cluster
            #puedo poner un filtro de probabilidad
            
            if maximo > umbral:
                if len(Ranking)==0:
                    Ranking.append([int(idimagen),maximo*10000,Listaordenada[idx].id])
                else:
                    if len(np.argwhere(np.array(Ranking,dtype="object")[:,0]==idimagen))==0:
                        Ranking.append([int(idimagen),maximo*10000,Listaordenada[idx].id])
                        
            ind_clust = np.argwhere(losclusters[:,0]==idimagen)
            id_clusters = losclusters[ind_clust,1]
            lista_id = np.array([lista.id for lista in Listaordenada])
            #Elimino el cluster de la lista y su probmax
            Listaordenada[idx].probmax = -1
            #Tambien marco la imagen y su probabilidad como ya utilizada
            indpos = np.argwhere(Listaordenada[idx].imagenes == idimagen)[0][0]
            #En todas las imagenes de ese cluster que sean igual a la idimagen
            Listaordenada[idx].proba[indpos] = -1
            
            #quito la imagen de otros posibles clusterings, 
            #quito la probabilidad maxima
            
            for j in range(len(id_clusters)):
                #encuentro la posicion de los otros cluster donde esta la imagen
                #busco cada cluster j 
                posicion = int(np.argwhere(lista_id == id_clusters[j]))
                #REVISAR ESTE CODIGO
                
                if posicion != idx:
                    #Encuentro las imagenes que son iguales en los otros clusters, pero el cluster no lo marco como
                    indpos = np.argwhere(Listaordenada[posicion].imagenes == idimagen)
                    #elimino de todos los otros clusters esa imagen poniendo la prob a -1
                    Listaordenada[posicion].proba[indpos] = -1
                    #Contamos cuantas imagenes elimino
                    cantidad = len(indpos)
                    Listaordenada[posicion].numimag = Listaordenada[posicion].numimag - cantidad
                    #Actualizo la probabilidad del cluster
                    probmax = Listaordenada[posicion].proba.max()
                    idx2 = Listaordenada[posicion].proba.argmax()
                    
                    if Listaordenada[posicion].probmax != -1: #solo actualizo si el cluster no esta eliminado
                        Listaordenada[posicion].probmax = probmax
                    #Actualizo que imagen es la que da esa probabilidad
                    Listaordenada[posicion].imagenmax = Listaordenada[posicion].imagenes[idx2]
                    
            indclus = np.argwhere(np.array([lista.probmax for lista in Listaordenada])>umbral)
            
            if len(indclus)==0:
                seguir = False
        Listaordenada = ActualizarProbabilidadesClusters(Listaordenada)
        numclusters = ClustersConImagenes(Listaordenada)
        
        if numclusters==0:
            continuar = False
        if (vueltas>=12) or (len(Ranking)>50):
            continuar = False
        vueltas += 1
        
    Listaordenada = VectorClusters
    
    return Listaordenada,Ranking

In [5]:
def metodoMediaEval(handles,runvisual):
    #Juntamos todos los datos
    handles.datosyqueriesynegativasorden = np.vstack((handles.negativas,handles.datosyqueriesorden))
    
    cantidad,tam = handles.datosyqueriesynegativasorden.shape
    cantidadpos,tam2 = handles.queries.shape
    cantidadneg,tam3 = handles.negativas.shape
    
    #Asigno con -1 a las neutras, con 0 a las negativas y con 1 a las positivas
    relevanciaOrdenInicial = np.ones((cantidad,1))*-1
    relevanciaOrdenInicial[:cantidadneg] = 0
    relevanciaOrdenInicial[cantidad-cantidadpos:] = 1
    
    lista_imag_rel = relevanciaOrdenInicial
    Rlocation = handles.Rlocation
    v_datos = handles.datosyqueriesynegativasorden[:,2:]
    
    if handles.ajuste=="logistic"
        #Ajustamos un modelo en R con los v_datos como matriz X de entrenamiento y lista_imag_rel como Y a predecir
        vect_prob,v_componentesModelo = AjusteMultiplesModelos(v_datos,lista_imag_rel,Rlocation,handles.ajuste)
        pesos = CalculoPesosModelo(handles.variances_pca,v_componentesModelo)
        proba_rel = np.matmul(vect_prob,pesos)
        proba_rel = proba_rel[cantidadneg:]
        identificadores = handles.datosyqueriesorden[:,0]
        prob_result = np.c_[identificadores,proba_rel]
        imag_prob = prob_result[:-cantidadpos]
        
    umbralprob = 0.2
    
    if runvisual==1:
        clusters = handles.datosclustervisual
    else:
        clusters = handles.datosclusters
    
    VectorClusters,Ranking = metodoOrdenarPorClusters(imag_prob,clusters,umbralprob)
    
    #Convierto Ranking en un array por comodidad
    Ranking = np.array(Ranking,dtype="object")
    
    res_tmp = handles.st
    img_text = np.c_[handles.st[:,0],handles.st[:,2]]
    
    handles.Ranking = Ranking
    
    #Guardamos Ranking en un fichero
    with open("Ranking.txt", "w",newline="") as f:
        writer = csv.writer(f)
        writer.writerows(Ranking)
    
    filpr, colpr = prob_result.shape
    score = []
    n_clus = []
    for i in range(filpr - cantidadpos):
        idimag = imag_prob[i,0]
        indice = np.argwhere(Ranking[:,0]==idimag)
        if not indice.size==0:
            score.append(indice[0][1])
            n_clus.append(Ranking[indice[0][1],2])
        else:
            score.append(1000)
            n_clus.append(-1) #No viene de ningun cluster
            
    score = np.array(score,dtype="object")
    n_clus = np.array(n_clus,dtype="object")
    
    #Creamos una matriz con las el nombre de las imagenes, su posicion en el ranking (1000) en caso de no aparecer y el cluster 
    #al que pertenecen (-1 si no estan en el ranking)
    prob_result = np.c_[(imag_prob[:,0],score,n_clus)]
    
    #Ordenamos esta matriz en base a la posicion del ranking y nos guardamos las posiciones que tenian
    
    ResultadoOrdenado = np.sort(score)
    aux = np.argsort(score)
    
    handles.datos = handles.datosorden[aux,:]
    
    #Añadimos las queries al final con sus scores
    filas_queries,col_queries = handles.queries.shape
    
    handles.datosyqueries = np.vstack((handles.datos,handles.queries))
    
    scoreQueries = np.ones(filas_queries)*1000
    
    #handles.imaProb = np.vstack((Ranking[:,1],scoreQueries))
    
    
    
    
    return handles


In [35]:
def saveMediaEvalResults(handles,fp):

    iteracion = 0

    if handles.algoritmo == 1:
        run_id = 'run1_visual'
    elif handles.algoritmo == 3:
        run_id = 'run3_multimedia'
    elif handles.algoritmo == 5:
        run_id = 'run5_relfeedback'

    sim = 1.0
    tam = len(handles.Ranking)
    minimo = min(50,tam)

    for i in range(minimo):
        rank = i
        fp.write(f"{handles.topic} {iteracion} {handles.Ranking[i,0]} {rank} {sim} {run_id}\n")
        sim = round(sim-0.01,2)